In [1]:
# TODO: look into nbextensions for Jupyter, specifically code_prettify

In [91]:
# suggestion: try to avoid nebulously named objects, e.g. requests --> r
import numpy as np
import requests as r
import pandas as pd
import json
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from time import sleep
from random import randint

In [2]:
# read in csv to dataframe
# TODO: look into os.path,join for handling relative file locations

datafile =  'spreadspoke_scores.csv'
df = pd.read_csv(datafile)

In [3]:
# Eliminate observations that do not have the data we need
df = df[(df.spread_favorite.notnull()) &
        (df.over_under_line.notnull()) &
        (df.score_home.notnull()) &
        (df.score_away.notnull()) &
        (df.weather_temperature.notnull()) &
        (df.weather_wind_mph.notnull())]


In [4]:
# drop unwanted columns
drop_cols = [ 'stadium_neutral']
for col in drop_cols:
    df.drop(col, axis=1, inplace=True)

In [5]:
# Only want post 1987 data to avoid last NFL strike
df = df[df.schedule_season > 1987]

# Fill in weather detail data
#df.weather_detail=df.weather_detail.fillna('CLEAR')- considered this but weather detail column was too unpopulated

In [6]:
## teamSwitch takes a string of the name of any football team from the past 30 years and returns the current abbreviation
def teamSwitch(full):
    abv = ''
    if full == 'New York Jets' or full == 'New York Giants':
        for let in range(len(full)):
            if let == 0:
                abv += full[let]
            elif full[let] == ' ':
                abv += full[let + 1]
    elif full.startswith('New ') or full.startswith(
            'Kans') or full[6:9] == 'Bay' or full.endswith('49ers'):
        for let in range(len(full)):
            if let == 0:
                abv += full[let]
            elif full[let] == ' ':
                abv += full[let + 1]
                return abv
    elif full.endswith('Oilers'):
        return "TEN"
    elif full.endswith('Cardinals'):
        return 'ARI'
    elif full.endswith('Rams'):
        return 'LAR'
    elif full.endswith('Chargers'):
        return 'LAC'
    elif full.endswith('Raiders'):
        return 'OAK'
    elif full[0:4] == 'Jack':
        return 'JAX'
    else:
        return full[0:3].upper()
    return abv

In [7]:
#weekNums takes the named weeks of the playoff rounds and converts them to ints
def weekNums(cell):
    if cell=='Division':
        cell= int(19)
    elif cell.upper()=='WILDCARD':
        cell=int(18)
    elif cell== 'Conference':
        cell=int(20)
    elif cell.upper()== 'SUPERBOWL':
        cell=int(21)
    else:
        cell=int(cell)
    return cell

In [8]:
#takes a pro football stadium and shows its zip code
def zipCode(cell):
    if cell==('Giants Stadium') or cell== ('MetLife Stadium'):
        cell= '07073'
    elif cell=='Lambeau Field':
        cell='54304'
    elif cell=='Arrowhead Stadium':
        cell='64129'
    elif cell== 'Soldier Field':
        cell='60605'
    elif cell==('Louisiana Superdome')or cell== ('Mercedes-Benz Superdome'):
        cell = '70112'
    elif cell=='Qualcomm Stadium':
        cell= '92108'
    elif cell=='Sun Life Stadium'or cell=='Hard Rock Stadium':
        cell='33056'
    elif cell== 'Ralph Wilson Stadium' or cell=='New Era Field':
        cell= '14127'
    elif cell== 'Georgia Dome' or cell=='Atlanta-Fulton County Stadium' :
        cell='30313'
    elif cell == 'Candlestick Park' or cell=='Levi\'s Stadium':
        cell= '95054'
    elif cell == 'University of Phoenix Stadium':
        cell= '85305'
    elif cell=='Oakland Coliseum':
        cell= '94621'
    elif cell== 'Bank of America Stadium':
        cell ='28202'
    elif cell== 'EverBank Field' or cell=='TIAA Bank Field':
        cell='32202'
    elif cell== 'FedEx Field' or cell== 'RFK Memorial Stadium':
        cell='20785'
    elif cell== 'Hubert H. Humphrey Metrodome' or cell=='Mall of America Field' or cell=='U.S. Bank Stadium':
        cell='55414'
    elif cell== 'Edward Jones Dome':  
        cell='63101'
    elif cell=='Texas Stadium' or cell=='Cowboys Stadium' or cell=='AT&T Stadium':
        cell='76011'
    elif cell== 'M&T Bank Stadium':
        cell= '21230'
    elif cell== 'RCA Dome' or cell=='Lucas Oil Stadium':
        cell='46225'
    elif cell== 'Raymond James Stadium' or cell=='Houlihan\'s Stadium':  
        cell='33607'
    elif cell=='FirstEnergy Stadium' or cell=='Cleveland Municipal Stadium':
        cell='44114'  
    elif cell== 'Nissan Stadium'or cell=='LP Stadium':  
        cell='37213'
    elif cell=='Paul Brown Stadium' or cell=='Cinergy Field':
        cell='45202'
    elif cell== 'Heinz Field' or cell=='Three Rivers Stadium':
        cell='15212'
    elif cell== 'Sports Authority Field at Mile High' or cell=='Mile High Stadium':
        cell='80204'
    elif cell== 'Gillette Stadium' or cell=='Foxboro Stadium':  
        cell='02035'
    elif cell=='CenturyLink Field' or cell=='Seattle Kingdome':
        cell='98134'
    elif cell=='Ford Field' or cell=='Pontiac Silverdome':
        cell='48226'  
    elif cell=='Lincoln Financial Field' or cell=='Veterans Stadium':
        cell='19148'
    elif cell=='Los Angeles Memorial Coliseum':
        cell='90037'   
    elif cell=='Anaheim Stadium':
        cell='92806' 
    elif cell=='Sun Devil Stadium':
        cell='85287' 
    elif cell=='Wembley Stadium':
        cell=''
    elif cell=='Husky Stadium':
        cell='98195'
    elif cell== 'TCF Bank Stadium':
        cell= '55455'
    elif cell=='Memorial Stadium (Baltimore)':
        cell='21218'
    elif cell=='StubHub Center':
        cell= '90746'
    elif cell=='Memorial Stadium (Champaign)':
        cell='61820'
    elif cell=='Vanderbilt Stadium':
        cell='37203'
    elif cell=='Rogers Centre':
        cell=''
    elif cell=='Tiger Stadium (LSU)':
        cell='70803'
    elif cell=='Busch Memorial Stadium':
        cell='63102'
    elif cell=='Twickenham Stadium':
        cell=''
    elif cell=='Alamo Dome':
        cell='78203'
    elif cell=='Estadio Azteca':
        cell=''
    elif cell=='Tottenham Stadium':
        cell=''
    elif cell==('Reliant Stadium')or cell== ('NRG Stadium')  or cell == ('Houston Astrodome'):
        cell = '77054'
    elif cell==('Memorial Stadium (Clemson)'):
        cell = '29634'
    elif cell==('Liberty Bowl Memorial Stadium'):
        cell = '38104'
    return cell

In [9]:
# takes a pro football stadium and return its coordinates
# TODO: take a look at a geocoding API for converting an address into a lat/lng, e.g. Google, HERE, Mapbox, ESRI, Nominatim

def stadCord(cell):
    if cell==('Giants Stadium') or cell== ('MetLife Stadium'):
        cell= (40.8135 , -74.0745)
    elif cell=='Lambeau Field':
        cell=(44.5013, -88.0622)
    elif cell=='Arrowhead Stadium':
        cell=(39.0489, -94.4839)
    elif cell== 'Soldier Field':
        cell=(41.8623, -87.6167)
    elif cell==('Louisiana Superdome')or cell== ('Mercedes-Benz Superdome'):
        cell = (29.9511, -90.0812)
    elif cell=='Qualcomm Stadium':
        cell= (32.7764, -117.1183)
    elif cell=='Sun Life Stadium'or cell=='Hard Rock Stadium':
        cell=(25.9580, -80.2389)
    elif cell== 'Ralph Wilson Stadium' or cell=='New Era Field':
        cell= (42.7738, -78.7870)
    elif cell== 'Georgia Dome' or cell=='Atlanta-Fulton County Stadium' :
        cell=(33.7577, -84.4008)
    elif cell == 'Candlestick Park' or cell=='Levi\'s Stadium':
        cell= (37.7136, -122.3861)
    elif cell == 'University of Phoenix Stadium':
        cell= (33.5276, -112.2626)
    elif cell=='Oakland Coliseum':
        cell= (37.7516, -122.2005)
    elif cell== 'Bank of America Stadium':
        cell =(35.2258, -80.8528)
    elif cell== 'EverBank Field' or cell=='TIAA Bank Field':
        cell=(30.3238, -81.6375)
    elif cell== 'FedEx Field' or cell== 'RFK Memorial Stadium':
        cell=(38.9077, -76.8644)
    elif cell== 'Hubert H. Humphrey Metrodome' or cell=='Mall of America Field' or cell=='U.S. Bank Stadium':
        cell=(44.9738, -93.2580)
    elif cell== 'Edward Jones Dome':  
        cell=(38.6327, -90.1886)
    elif cell=='Texas Stadium' or cell=='Cowboys Stadium' or cell=='AT&T Stadium':
        cell=(32.84, -96.911)
    elif cell== 'M&T Bank Stadium':
        cell=(39.2780, -76.6227)
    elif cell== 'RCA Dome' or cell=='Lucas Oil Stadium':
        cell=(39.7636, -86.1633)
    elif cell== 'Raymond James Stadium' or cell=='Houlihan\'s Stadium':  
        cell=(27.9758, -82.5033)
    elif cell=='FirstEnergy Stadium' or cell=='Cleveland Municipal Stadium':
        cell=(41.5066, -81.6972)   
    elif cell== 'Nissan Stadium'or cell=='LP Stadium':  
        cell=(36.1663, -86.7713)
    elif cell=='Paul Brown Stadium' or cell=='Cinergy Field':
        cell=(39.095, -84.516)
    elif cell== 'Heinz Field' or cell=='Three Rivers Stadium':
        cell=(40.4466, -80.0158)
    elif cell== 'Sports Authority Field at Mile High' or cell=='Mile High Stadium':
        cell=(39.7438, -105.02)
    elif cell== 'Gillette Stadium' or cell=='Foxboro Stadium':  
        cell=(42.0909, -71.2643)
    elif cell=='CenturyLink Field' or cell=='Seattle Kingdome':
        cell=(47.5952, -122.3316)  
    elif cell=='Ford Field' or cell=='Pontiac Silverdome':
        cell=(42.34, -83.0455)  
    elif cell=='Lincoln Financial Field' or cell=='Veterans Stadium':
        cell=(39.9008, -75.1675) 
    elif cell=='Los Angeles Memorial Coliseum':
        cell=(34.0141, -118.2877)     
    elif cell=='Anaheim Stadium':
        cell=(33.8002, -117.8827)  
    elif cell=='Sun Devil Stadium':
        cell=(33.4263, -111.93)  
    elif cell=='Wembley Stadium':
        cell=(51.5558, -0.2797)
    elif cell=='Husky Stadium':
        cell=(47.6502, -122.3016)
    elif cell== 'TCF Bank Stadium':
        cell= (44.976, -93.225)
    elif cell=='Memorial Stadium (Baltimore)':
        cell=(39.3294, -76.6013)
    elif cell=='StubHub Center':
        cell=(33.864, -118.261)
    elif cell=='Memorial Stadium (Champaign)':
        cell=(40.0991, -88.2358)
    elif cell=='Vanderbilt Stadium':
        cell=(36.1441, -86.8088)
    elif cell=='Rogers Centre':
        cell=(43.6413, -79.3891)
    elif cell=='Tiger Stadium (LSU)':
        cell=(30.4119, -91.1855)
    elif cell=='Busch Memorial Stadium':
        cell=(38.6238, -90.1925)
    elif cell=='Twickenham Stadium':
        cell=(51.4561, -0.3416)
    elif cell=='Alamo Dome':
        cell=(29.4169, -98.4788)
    elif cell=='Estadio Azteca':
        cell=(19.3029, -99.1504)
    elif cell=='Tottenham Stadium':
        cell=(51.6044, -0.0663)
        
    elif cell==('Reliant Stadium')or cell == ('NRG Stadium') or cell == ('Houston Astrodome'):
        cell = (29.6847, -95.4107)
    elif cell==('Memorial Stadium (Clemson)'):
        cell = (34.6788, -82.8432)
    elif cell==('Liberty Bowl Memorial Stadium'):
        cell = (35.1211, -89.9774)
        
    return cell

In [10]:
#changes the date format to be web scrape friendly
def dateSwitch(cell):
    cell = cell[6:] + cell[5] + cell[0:3] + cell[3:5]
    cell = cell.replace('/', '-')
    return cell

In [11]:
# TODO: read-up on the dangers of using .copy() with dataframes
# Suggestion: self-documenting code, e.g. instead of ppdf, use df_junk, df_test, df_copy...
ppdf = df.copy()

# apply teamSwitch to data
ppdf.team_home = ppdf.team_home.apply(teamSwitch)
ppdf.team_away = ppdf.team_away.apply(teamSwitch)

# create a column to show if the home team is favored
homefave = ppdf.team_home == ppdf.team_favorite_id
ppdf['home_fave'] = homefave

# create a column to show the total points scored by both teams
ppdf['total'] = ppdf.score_home + ppdf.score_away


In [12]:
# Add stadium coordinates
col = ppdf.copy()
col.stadium = col.stadium.apply(stadCord)
ppdf['coordinates'] = col.stadium
col = ppdf.copy()

# Add zipcode
col.stadium = col.stadium.apply(zipCode)
ppdf['zipcode'] = col.stadium

In [13]:
# reformat dates
# TODO: look into the efficiency of doing map vs apply vs ...?
ppdf.schedule_week = ppdf.schedule_week.apply(weekNums)
ppdf = ppdf[ppdf.zipcode != '']
ppdf.schedule_date = ppdf.schedule_date.apply(dateSwitch)

In [14]:
# Create a dict, result, to track how teams do compared to the spread. If favorite wins, the number is negative and  the number is positive if the underdog wins
# Initialize empty dict
result = {}

# loop through dataFrame index to populate dict with keys from the index  and values from columns
# TODO: look into how you could make the below more efficient... maybe a one-liner
for x in ppdf.index:
    if ppdf.loc[x]['home_fave'] == True:
        result[x] = ppdf.loc[x]['score_away'] - ppdf.loc[x]['score_home']
    else:
        result[x] = ppdf.loc[x]['score_home'] - ppdf.loc[x]['score_away']

In [15]:
#create new column for result
result = pd.Series(result)
ppdf['results'] = result
ppdf['results-spread'] = ppdf['results'] - ppdf['spread_favorite']

#using the new results column, create a bolean series to become a new column to indicate if the spread ws covered
cover = ppdf.results < ppdf.spread_favorite
ppdf['favorite_cover'] = cover

#create a boolean series to make a column to indicate if the over was reached
ppdf['over_under_line'] = ppdf['over_under_line'].astype(float)
over = ppdf['total'] > ppdf['over_under_line']
ppdf['over_cover'] = over
over_diff = ppdf.total- ppdf.over_under_line 
ppdf['over_diff'] = over_diff

#rearrange columns for more readable DataFrame
ppdf = ppdf[[
    'schedule_date', 'schedule_season', 'schedule_week', 'team_home',
    'score_home', 'score_away', 'team_away', 'team_favorite_id',
    'spread_favorite', 'results', 'results-spread', 'favorite_cover',
    'over_under_line', 'total', 'over_diff', 'over_cover',
    'weather_temperature', 'weather_wind_mph', 'stadium', 'coordinates',
    'zipcode', 'weather_detail'
]]

In [113]:
#Test path to see if we are getting the right data
pracdf = ppdf.copy()
page = r.get('https://www.almanac.com/weather/history/zipcode/' +
             pracdf.loc[5402]['zipcode'] + '/' +
             pracdf.loc[5402]['schedule_date'])
soup = BeautifulSoup(page.content, 'html.parser')
prcp = soup.find('tr', class_='weatherhistory_results_datavalue prcp')
data = prcp.find('td')
nxt = data.find('p')
point = nxt.find(class_='value')
point.get_text()

u'0.00'

In [95]:
pracdf.loc[4685]['zipcode'] + '/' +pracdf.loc[4685]['schedule_date']

4673    55414/1988-12-19
4675    55414/1988-12-26
dtype: object

In [150]:
pracdf.weather_detail

4450      NaN
4451      NaN
4452      NaN
4453      NaN
4454     DOME
4455      NaN
4456     DOME
4457      NaN
4458      NaN
4459      NaN
4460     DOME
4461      NaN
4462      NaN
4463      NaN
4464      NaN
4465      NaN
4466      NaN
4467      NaN
4468      NaN
4469     DOME
4470     DOME
4471      NaN
4472     DOME
4473      NaN
4474      NaN
4475     DOME
4476      NaN
4477      NaN
4478      NaN
4479      NaN
         ... 
12370    DOME
12371     NaN
12372     NaN
12373    DOME
12374    DOME
12375    DOME
12376     NaN
12377     NaN
12378    DOME
12379     NaN
12380     NaN
12381     NaN
12382     NaN
12383    Rain
12384     NaN
12385     NaN
12386     NaN
12387     NaN
12388    DOME
12389     NaN
12390     NaN
12391    DOME
12392     NaN
12393    DOME
12394     NaN
12395     NaN
12396     NaN
12397     NaN
12398     NaN
12399     NaN
Name: weather_detail, Length: 7692, dtype: object

In [19]:
for i, col in enumerate(pracdf.columns):
    print('col {}: {}'.format(i, col))

col 0: schedule_date
col 1: schedule_season
col 2: schedule_week
col 3: team_home
col 4: score_home
col 5: score_away
col 6: team_away
col 7: team_favorite_id
col 8: spread_favorite
col 9: results
col 10: results-spread
col 11: favorite_cover
col 12: over_under_line
col 13: total
col 14: over_diff
col 15: over_cover
col 16: weather_temperature
col 17: weather_wind_mph
col 18: stadium
col 19: coordinates
col 20: zipcode
col 21: weather_detail


In [164]:
#create dic to store values


count=0

# loop through index and populate dict with precipitation data
# TODO: look into how to efficiently loop over a dataframe
for i in pracdf.index:
    # temp debug continue
    #if pracdf.loc[i]['team_home'] != 'DET':
    #    continue
    
    if count>50:
        sleep(20)
        count=0
        
        
    if pracdf.loc[i]['weather_detail'] =='DOME':
        precipitation[i] = 0
    else:
        
        print('Working on: {} {} -- {} v {}'.format(i, pracdf.loc[i]['schedule_date'], pracdf.loc[i]['team_home'], pracdf.loc[i]['team_away']))
        
        page = r.get('https://www.almanac.com/weather/history/zipcode/' +
                     pracdf.loc[i]['zipcode'] + '/' +
                     pracdf.loc[i]['schedule_date'])
        soup = BeautifulSoup(page.content, 'html.parser')
        prcp = soup.find('tr', class_='weatherhistory_results_datavalue prcp')
        if prcp== None:
            count+=1
            sleep(10)
            
            page = r.get('https://www.almanac.com/weather/history/zipcode/' +
                     pracdf.loc[i]['zipcode'] + '/' +
                     pracdf.loc[i]['schedule_date'])
            soup = BeautifulSoup(page.content, 'html.parser')
            prcp = soup.find('tr', class_='weatherhistory_results_datavalue prcp')
            if prcp== None:
                print(pracdf.loc[i]['zipcode'], 
                  pracdf.loc[i]['schedule_date'], 
                  'Nonetype Detected')
                precipitation[i] = -999
                
            else:
                data = prcp.find('td')
                nxt = data.find('p')
    #print('---->>>>>', )
    #print('---->>>>>', nxt.find(class_='nullvalue'))
                if nxt.get_text() == 'No data.':
                    print('WARNING: no precipitation data for date and game')
                    precipitation[i] = -999
                else:
                    point = nxt.find(class_='value')
                #print('---->>>>>', point)#, point.get_text())
                    precipitation[i] = point.get_text()
                    print(precipitation[i], count)
        else:
            data = prcp.find('td')
            nxt = data.find('p')
            if nxt.get_text() == 'No data.':
                
                precipitation[i] = -999
            else:
                point = nxt.find(class_='value')
               
                precipitation[i] = point.get_text()
                print(precipitation[i], count)
             
            
            
precipitation = pd.Series(precipitation)

Working on: 12366 2018-12-16 -- SF v SEA
(u'0.00', 0)
Working on: 12367 2018-12-17 -- CAR v NO
(u'0.00', 0)
Working on: 12368 2018-12-22 -- LAC v BAL
(u'0.00', 0)
Working on: 12369 2018-12-22 -- TEN v WAS
(u'0.03', 0)
Working on: 12371 2018-12-23 -- CAR v ATL
(u'0.00', 0)
Working on: 12372 2018-12-23 -- CLE v CIN
(u'0.10', 0)
Working on: 12376 2018-12-23 -- MIA v JAX
(u'0.00', 0)
Working on: 12377 2018-12-23 -- NE v BUF
(u'0.02', 0)
Working on: 12379 2018-12-23 -- NYJ v GB
(u'0.01', 0)
Working on: 12380 2018-12-23 -- PHI v HOU
(u'0.09', 0)
Working on: 12381 2018-12-23 -- SF v CHI
(u'0.00', 0)
Working on: 12382 2018-12-23 -- SEA v KC
(u'0.44', 0)
Working on: 12383 2018-12-24 -- OAK v DEN
(u'0.00', 0)
Working on: 12384 2018-12-30 -- BAL v CLE
(u'0.00', 0)
Working on: 12385 2018-12-30 -- BUF v MIA
(u'0.08', 0)
Working on: 12386 2018-12-30 -- DEN v LAC
(u'0.00', 0)
Working on: 12387 2018-12-30 -- GB v DET
(u'0.00', 0)
Working on: 12389 2018-12-30 -- KC v OAK
(u'0.00', 0)
Working on: 12390 

In [221]:
# 7:40:00 10673    Working on: 11139 2014-10-05 -- DAL v HOU
ppdf['precipitation']=precipitation
ppdf=ppdf.loc[ppdf.precipitation>-1]


In [193]:
for i in ppdf.index:
    if ppdf.loc[i,'weather_detail']=='DOME':
        ppdf.loc[i,'precipitation']=0
        


In [219]:
ppdf.notna().sum()

schedule_date          7375
schedule_season        7375
schedule_week          7375
team_home              7375
score_home             7375
score_away             7375
team_away              7375
team_favorite_id       7375
spread_favorite        7375
results                7375
results-spread         7375
favorite_cover         7375
over_under_line        7375
total                  7375
over_diff              7375
over_cover             7375
weather_temperature    7375
weather_wind_mph       7375
stadium                7375
coordinates            7375
zipcode                7375
weather_detail         1979
precipitation          7375
dtype: int64

In [220]:
ppdf.to_pickle('clean_data.pkl')